In [1]:
import pandas as pd

In [7]:
import json
from typing import Dict, List
from pydantic import BaseModel

In [16]:
data_path = '/home/liujunwen/MAKER_BRB/data/demodata.json'
data_list = []
with open(data_path,'r')as f:
    for line in f:
        _data = json.loads(line.strip())
        print(_data)
        data_list.append(_data)

{'id': 0, 'features': {'V1': {'A1': 0.3, 'B1': 0.7, 'C1': 0, 'D1': 0}, 'V2': {'A2': 0.0, 'B2': 0.5, 'C2': 0.5, 'D2': 0}, 'V3': {'A3': 0.0, 'B3': 1}, 'h': 0}}
{'id': 1, 'features': {'V1': {'A1': 0.0, 'B1': 0.5, 'C1': 0.5, 'D1': 0}, 'V2': {'A2': 1, 'B2': 0, 'C2': 0, 'D2': 0}, 'V3': {'A3': 1, 'B3': 0}, 'h': 1}}
{'id': 2, 'features': {'V1': {'A1': 0.0, 'B1': 0.0, 'C1': 1, 'D1': 0}, 'V2': {'A2': 0, 'B2': 1, 'C2': 0, 'D2': 0}, 'V3': {'A3': 1, 'B3': 0}, 'h': 1}}
{'id': 3, 'features': {'V1': {'A1': 0.0, 'B1': 0.0, 'C1': 1.0, 'D1': 0}, 'V2': {'A2': 0, 'B2': 1, 'C2': 0, 'D2': 0}, 'V3': {'A3': 0, 'B3': 1}, 'h': 0}}
{'id': 4, 'features': {'V1': {'A1': 0.0, 'B1': 0.0, 'C1': 0.0, 'D1': 1}, 'V2': {'A2': 0, 'B2': 0, 'C2': 0, 'D2': 1}, 'V3': {'A3': 1, 'B3': 0}, 'h': 1}}


In [1]:
'''
一条数据
{
        "id": 0,
        "features": {
            "v1": {
                "A1": 0.3,
                "B1": 0.7,
                "C1": 0,
                "D1": 0
            },
            "V2": {
                "A2": 0.0,
                "B2": 0.5,
                "C2": 0.5,
                "D2": 0
            },
            "V3": {
                "A3": 0.0,
                "B3": 1
            },
            "h": 0
        }
    }
'''
def get_phvq(data_list,h,index_list:List):
    '''
    data_list每项为一条数据
    v对应1级指标，q对应2级指标
    统计sum(h,v,q)/sum(v,q)
    '''
    h_set = get_h_set()
    C_h_set = {}
    for _h in h_set:
        C_h_set[_h] = 0

    C_h_v_q = 0
    C_v_q = 0
    for _data in data_list:
        _features = _data['features']
        _c_v_q = 1
        for index in index_list:
            v = index['v']
            q = index['q']
            index_v = _features.get(v, None)
            if not index_v:
                print('index: %s not exist'%v)
                print(_data)
                break
            _c_v_q *= index_v.get(q,0)
        C_v_q += _c_v_q
        ???
        if _features['h'] == h:
            C_h_v_q += _c_v_q
    if C_v_q == 0:
        return 0
    return C_h_v_q/C_v_q

def get_interrelation_h_vq(data_list,h,index_list:List):
    p_product = 1
    for index in index_list:
        _p = get_phvq(data_list,h,[index])
        if _p == 0:
            return 0
        p_product *= _p
    p_multi = get_phvq(data_list,h,index_list)
    return p_multi/p_product

def get_h_set(data_list):
    return set([data['features']['h'] for data in data_list ])

def get_features():
    return {'V1':["A1","B1","C1","D1"],'V2':["A2","B2","C2","D2"],'V3':["A3","B3"]}


NameError: name 'List' is not defined

In [17]:
get_phvq(data_list=data_list,h=0,index_list= [{'v':'V1','q':'A1'},{'v':'V2','q':'B2'}])

1.0

In [18]:
get_interrelation_h_vq(data_list=data_list,h=0,index_list= [{'v':'V1','q':'A1'},{'v':'V2','q':'B2'}])

1.6666666666666667

In [6]:
get_features()

{'V1': {'A1', 'B1', 'C1', 'D1'},
 'V2': {'A2', 'B2', 'C2', 'D2'},
 'V3': {'A3', 'B3'}}

In [7]:
get_h_set(data_list)

{0, 1}

In [19]:
import torch.nn 

In [13]:
def get_features():
    return {'V1':["A1","B1","C1","D1"],'V2':["A2","B2","C2","D2"],'V3':["A3","B3"]}
def get_h_set(data_list):
    return set([data['features']['h'] for data in data_list ])


In [3]:
get_features()

{'V1': ['A1', 'B1', 'C1', 'D1'],
 'V2': ['A2', 'B2', 'C2', 'D2'],
 'V3': ['A3', 'B3']}

In [11]:
class Feature_info(BaseModel):
    h:str
    v:str
    q:str

class Feature(BaseModel):
    feature_id :int 
    feature_info:List[Feature_info]

In [24]:

def encode_features(feature_dict:Dict, h_set):
    '''
    对所有特征进行编码
    '''
    feature_list = []
    feature_id = 0
    for h in h_set:
        for v, q_list in feature_dict.items():
            
            for q in q_list:
                feature_info = Feature_info(h=h,v=v,q=q)
                feature = Feature(feature_id=feature_id, feature_info=[feature_info])
                feature_list.append(feature)
                feature_id += 1
    return feature_list


In [26]:
def find_feature(feature_info:List[Feature_info], feature_list:List[Feature]):
    for feature in feature_list:
        if feature_info == feature.feature_info:
            return feature


In [27]:
find_feature([Feature_info(h='0', v='V1', q='B1')],encode_features(get_features(),get_h_set(data_list)))

Feature(feature_id=1, feature_info=[Feature_info(h='0', v='V1', q='B1')])

In [25]:
encode_features(get_features(),get_h_set(data_list))

[Feature(feature_id=0, feature_info=[Feature_info(h='0', v='V1', q='A1')]),
 Feature(feature_id=1, feature_info=[Feature_info(h='0', v='V1', q='B1')]),
 Feature(feature_id=2, feature_info=[Feature_info(h='0', v='V1', q='C1')]),
 Feature(feature_id=3, feature_info=[Feature_info(h='0', v='V1', q='D1')]),
 Feature(feature_id=4, feature_info=[Feature_info(h='0', v='V2', q='A2')]),
 Feature(feature_id=5, feature_info=[Feature_info(h='0', v='V2', q='B2')]),
 Feature(feature_id=6, feature_info=[Feature_info(h='0', v='V2', q='C2')]),
 Feature(feature_id=7, feature_info=[Feature_info(h='0', v='V2', q='D2')]),
 Feature(feature_id=8, feature_info=[Feature_info(h='0', v='V3', q='A3')]),
 Feature(feature_id=9, feature_info=[Feature_info(h='0', v='V3', q='B3')]),
 Feature(feature_id=10, feature_info=[Feature_info(h='1', v='V1', q='A1')]),
 Feature(feature_id=11, feature_info=[Feature_info(h='1', v='V1', q='B1')]),
 Feature(feature_id=12, feature_info=[Feature_info(h='1', v='V1', q='C1')]),
 Feature(